In [ ]:
import pyspark.sql.functions as f
import pandas as pd
import json
from datetime import timedelta, datetime

In [ ]:
startdate = (datetime.now() - timedelta(1)).strftime("%Y-%m-%d")
enddate = (datetime.now() - timedelta(1)).strftime("%Y-%m-%d")
dates = [startdate,enddate]
dates

['2024-08-20', '2024-08-20']

Reading JSON files

In [ ]:
# Check the last JSON data in directory
source_path = "s3a://datalake-eu-central-1/data/events"
dbutils.fs.ls(source_path)[-2:-1]

In [ ]:
# Reading JSON dataset as a bulk
yesterday = (datetime.now() - timedelta(1)).strftime("%Y-%m-%d")

df = (
    spark.read
    .option('basePath', f"{source_path}")   # to read the partitioning field : dt col
    .json(f"{source_path}/dt={yesterday}")
    .limit(5)
    )

df.display()

Reading and Writing Delta

In [ ]:
# Reading Delta dataset as a bulk
file_path = "s3://zalando-datalake-binary/test_raw_data"
raw_df = (
        spark.read
        .format("delta")
        .load(file_path)
        .withColumn("source", f.lit("raw"))
        .filter(f.col("dt").between(*dates))
        .limit(4)
        )

raw_df.display()

In [ ]:
# Write as a bulk to Delta Lake
save_path = "/team/test_target_data"

(
    raw_df
    .write.format("delta")
    .mode("overwrite")
    .partitionBy("dt")
    .option("replaceWhere", f"dt >= '{start_date}' AND dt <= '{end_date}'")
    .save(save_path)
    
)